In [22]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
from torchvision import transforms
import random
from scipy.signal import convolve2d
import numpy as np

In [24]:
# define model
def mask_generator():
    random_index = np.random.randint(5,10)
    random_image = np.random.randint(2, size=(1,28,28)).squeeze().astype(np.float32)
    filter = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16
    for i in range(random_index):
        random_image = convolve2d(random_image, filter, mode='same', boundary='symm')
    mask = (random_image > 0.5).astype(np.float32)
    return mask

def negative_data_generator(image_batch):
    idxs = torch.randperm(image_batch.shape[0])
    x = image_batch[idxs]
    mask = torch.from_numpy(mask_generator())
    hybrid_images = x * mask + (1-mask) * x
    return hybrid_images

class FFLayer(nn.Module):
  def __init__(self,input_dim,output_dim,threshold=2.0,epochs=100):
      super().__init__()
      self.lin = nn.Linear(input_dim,output_dim)
      self.relu = nn.ReLU()
      self.optim = torch.optim.Adam(self.parameters(),lr=0.03)
      self.epochs = epochs
      self.threshold = threshold

  def forward(self,x):
    # x has shape B x D
    x = x / (x.norm(p=2,dim=1,keepdim=True)+1e-4)
    x = self.lin(x)
    return self.relu(x)

  def forward_forward(self,x_pos,x_neg):
    # x has shape B x h x w
    for epoch in range(self.epochs):
      h_pos = self.forward(x_pos)
      h_neg = self.forward(x_neg)
      good_pos = (self.threshold-h_pos**2).mean(dim=1)
      good_neg = (h_neg**2-self.threshold).mean(dim=1)
      self.optim.zero_grad()
      loss = torch.log(1+torch.exp(torch.cat([good_pos,good_neg]))).mean()
      loss.backward()
      self.optim.step()

    return self.forward(x_pos).detach(), self.forward(x_neg).detach()


class FF(nn.Module):
  def __init__(self,input_dim, hidden_dims,device,threshold=2.0,epochs=100):
      super().__init__()
      self.epochs = epochs
      self.threshold = threshold
      self.layers = []
      self.layers.append(FFLayer(input_dim,hidden_dims[0],threshold,epochs).to(device))
      for i in range(1,len(hidden_dims)):
        self.layers.append(FFLayer(hidden_dims[i-1],hidden_dims[i],threshold,epochs).to(device))

  def train(self,x_pos, x_neg):
    h_pos = x_pos.reshape((x_pos.shape[0],-1))
    h_neg = x_neg.reshape((x_pos.shape[0],-1))
    for layer in self.layers:
      h_pos, h_neg = layer.forward_forward(h_pos,h_neg)

  def test(self,x_pos,x_neg):
    with torch.no_grad():
      h_pos = x_pos.reshape((x_pos.shape[0],-1))
      h_neg = x_neg.reshape((x_pos.shape[0],-1))
      losses = []
      for layer in self.layers:
        h_pos = layer.forward(h_pos)
        h_neg = layer.forward(h_neg)
        good_pos = (self.threshold-h_pos**2).mean(dim=1)
        good_neg = (h_neg**2-self.threshold).mean(dim=1)
        loss = torch.log(1+torch.exp(torch.cat([good_pos,good_neg]))).mean()
        losses.append(loss.item())

      final_loss = np.array(losses).mean()
      return final_loss

  def inference(self,x):
    with torch.no_grad():
      x = x.reshape((x.shape[0],-1))
      for layer in self.layers:
        x = layer.forward(x)
      return x

In [25]:
# load dataset
transform = Compose([ToTensor()])
trainset = MNIST('.',train=True,download=True,transform=transform)
testset = MNIST('.',train=False, download=True,transform=transform)

batch_size = 1024
trainLoader = DataLoader(trainset,batch_size,shuffle=True)
testLoader = DataLoader(testset,batch_size,shuffle=True)

In [26]:
#train FF
input_dim = 784
hidden_dims = [512,512,512,512]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FF(input_dim, hidden_dims,device,epochs=100).to(device)

for x_pos,_ in tqdm(trainLoader):
  x_neg =  negative_data_generator(x_pos)
  x_neg = x_neg.to(device)
  x_pos = x_pos.to(device)
  model.train(x_pos,x_neg)

100%|██████████| 59/59 [00:52<00:00,  1.12it/s]


In [18]:
# test FF
testLoss = 0.0
for x_pos,_ in tqdm(testLoader):
  x_neg =  negative_data_generator(x_pos)
  x_neg = x_neg.to(device)
  x_pos = x_pos.to(device)
  testLoss += model.test(x_pos,x_neg)
testLoss = testLoss / len(testLoader)

trainLoss = 0.0
for x_pos,_ in tqdm(trainLoader):
  x_neg =  negative_data_generator(x_pos)
  x_neg = x_neg.to(device)
  x_pos = x_pos.to(device)
  trainLoss += model.test(x_pos,x_neg)
trainLoss = trainLoss / len(trainLoader)
print(f'train loss: {trainLoss}')
print(f'test loss: {testLoss}')

100%|██████████| 59/59 [00:06<00:00,  8.62it/s]

train loss: 0.6931644491219925
test loss: 0.693164199590683


In [27]:
torch.save(model.state_dict(), './FF.pth')

In [31]:
# define classifier
class LinearClassifier(nn.Module):
  def __init__(self,input_dim, num_classes, hidden_dims,device,threshold=2.0,epochs=100):
     super().__init__()
     self.ff = FF(input_dim, hidden_dims,device,threshold=2.0,epochs=100).to(device)
     self.ff.load_state_dict(torch.load('./FF.pth'))
     self.lin = nn.Linear(hidden_dims[-1],num_classes)

  def forward(self,x):
    x = self.ff.inference(x)
    x = self.lin(x)
    return x

با دیتای منفی و هزینه که شبکه آموزش داده شده، در عمل یاد میگیرد که ساختار ارقام را کد کند. جون تصویر منفی ساختار نامنظمی دارد درنتیجه با کم کردن حساسایت روی آن ها و افزایش حاسایت روی تصویر واقعی یاد میگیرد که  که اعداد تک رقمی چگونه نوشته میشوند. سپس با این بردار ویژگی آنها را طبقه بندی میکند.

چون هدف این بوده که
طبقه بند خطی باشد چون مان یک بردار ویژگی یاد گرفتیم و امنتظار داریم که به خوبی ارقام را تشخیص ندهد یعنی بتواند با مرزهای خطی ارقام را جدا کند. اگر بخواهیم از لایه های غیر خطی استفاده کنیم نمی توان کارایی روش غیر نظارتی را سنجید.








پیاده سازی کلی مانند قسمت قبل است فقط اینجا با آموزش شبکه FF ان را در حالت eval قرار داده و تابع خطی را آزمایش میکنیم.

In [34]:
# train classifier
input_dim = 784
num_classes = 10
hidden_dims = [512,512,512,512]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = LinearClassifier(input_dim,num_classes,hidden_dims,device).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(classifier.parameters(),lr=0.03)

losses = []
for epoch in range(100):
  current_loss = 0.0
  for image,target in tqdm(trainLoader):
    image = image.to(device)
    target = target.to(device)
    logits = classifier(image)
    optimizer.zero_grad()
    loss = criterion(logits,target)
    loss.backward()
    optimizer.step()
    current_loss += loss.item()/len(trainLoader)

  print(f'epoch {epoch}: loss = {current_loss}')



100%|██████████| 59/59 [00:07<00:00,  7.98it/s]


epoch 0: loss = 2.041261945740651


100%|██████████| 59/59 [00:07<00:00,  8.00it/s]


epoch 1: loss = 1.6143200639951027


100%|██████████| 59/59 [00:07<00:00,  7.75it/s]


...

epoch 97: loss = 0.3534873456267987


100%|██████████| 59/59 [00:06<00:00,  9.27it/s]


epoch 98: loss = 0.35268126150309037


100%|██████████| 59/59 [00:07<00:00,  8.42it/s]

epoch 99: loss = 0.35201816387095697


In [35]:
torch.save(classifier.state_dict(), './classifier.pth')

In [39]:
# test classifier
input_dim = 784
num_classes = 10
hidden_dims = [512,512,512,512]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = LinearClassifier(input_dim,num_classes,hidden_dims,device).to(device)
classifier.load_state_dict(torch.load('./classifier.pth'))


losses = []

acc_test = 0.0
for image,target in tqdm(testLoader):
  image = image.to(device)
  target = target.to(device)
  logits = classifier(image)
  idxs = logits.argmax(dim=1)
  acc_test += (idxs == target).sum().item()/len(target)

acc_test /= len(testLoader)

acc_train = 0.0
for image,target in tqdm(trainLoader):
  image = image.to(device)
  target = target.to(device)
  logits = classifier(image)
  idxs = logits.argmax(dim=1)
  acc_train += (idxs == target).sum().item()/len(target)

acc_train /= len(trainLoader)

print(f'Train accuracy: {acc_train*100}%')
print(f'Test accuracy: {acc_test*100}%')

100%|██████████| 59/59 [00:06<00:00,  9.16it/s]

Train accuracy: 89.65261903434434%
Test accuracy: 89.83458227040816%
